## 1. Imports

In [ ]:
import os
import glob
import sys
from pathlib import Path

import PIL.Image

# plots and images
import matplotlib.pyplot as plt

# Pytorch
import torch

In [ ]:
# If we want to use google drive
_USE_G_DRIVE = False

In [ ]:
# Mount google drive
base_path = "../"
if _USE_G_DRIVE:
  from google.colab import drive

  drive.mount("/content/drive/")
  base_path = Path('/content/drive/My Drive/Bridges-Crack-Detection/')
  sys.path.append(str(base_path))

# import our src package
sys.path.append(str("../"))
sys.path.append(str("../src/"))

#### Import model and utility functions

In [ ]:
from src.utils.utils import init_random, get_device
from src.utils.model import get_model
from src.utils.trainer import inference

## 2. Variables definition

### General Variables

In [ ]:
# Path for the images we want to classify
DS_PATH = os.path.join(base_path, "Dataset/Test/")
#------------------------------------------------
# Saved model path
MODEL_SAVE_PATH= os.path.join(base_path, "SavedWeights/trained_model.pt")
#------------------------------------------------

print("----- Paths -----")
print("Path Dataset To Classify:", str(DS_PATH))
print("Path Model Params:", str(MODEL_SAVE_PATH))

### Initialization

In [ ]:
# set deterministic to true to have deterministic results, set it to false to have faster inference
deterministic_behaviour = False
seed = 6788
init_random(deterministic_behaviour, seed)
device = get_device()

## 3. Load the model

In [ ]:
# Load the model into the device
model = get_model(device, verbose=True)
# Load trained weights
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
# Set the model to evaluation mode
model.eval()
print("Model Loaded")

## 4. Inference phase

### Prepare inference data

In [ ]:
# Load the dataset
data_to_predict = [o for o in glob.glob(os.path.join(DS_PATH, "*"))]
#-------------------------------------------------------

print("Number of images to classify:", len(data_to_predict))

### Prediction method

In [ ]:
def predict_image(model, path, device):
  img = PIL.Image.open(path)
  output = inference(model, path, device)
  return img, output

### Print images and their relative predictions

In [ ]:
# Set number of plots to show
N_ROWS = 2 # Must be > 1
N_COLUMNS = 3 # Must be > 1
N_IMAGES_TO_PLOT = N_ROWS * N_COLUMNS
# If we have more images only N_IMAGES_TO_PLOT will be shown, for the others we will have only the print

plots_height = 10
plots_width = 10

fig,a = plt.subplots(N_ROWS, N_COLUMNS)
fig.set_figheight(plots_height)
fig.set_figwidth(plots_width)

i = j = 0
cont = 0
for img_path in data_to_predict:
  output_image, result = predict_image(model, img_path, device)
  result = "Normal" if result == 0 else "Crack" if result == 1 else "Spalling"
  print(f"The image was classified as: {result}\n")
  if cont < N_IMAGES_TO_PLOT:
    a[i][j].imshow(output_image)
    a[i][j].set_title(result)
    j += 1
    if j == N_COLUMNS:
      j = 0
      i += 1
  cont += 1